In [1]:
import pandas as pd
import numpy as np

In [6]:
df = pd.read_csv(r"C:\Users\Urvashi\Desktop\New folder\Urvashi\Datasets\Amazon_Unlocked_Mobile.csv")

In [7]:
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 413840 entries, 0 to 413839
Data columns (total 6 columns):
Product Name    413840 non-null object
Brand Name      348669 non-null object
Price           407907 non-null float64
Rating          413840 non-null int64
Reviews         413778 non-null object
Review Votes    401544 non-null float64
dtypes: float64(2), int64(1), object(3)
memory usage: 18.9+ MB


In [9]:
df.isnull().sum()

Product Name        0
Brand Name      65171
Price            5933
Rating              0
Reviews            62
Review Votes    12296
dtype: int64

In [10]:
df["Rating"].value_counts()

5    223605
1     72350
4     61392
3     31765
2     24728
Name: Rating, dtype: int64

In [15]:
# Data Cleaning : Drop the rows with null values

df.dropna(inplace = True)
df = df[df["Rating"] !=3] # drop the data where Rating ==3

# create a new column 'Positively rated' where, if rating is more than 3, we put 1 else 0

df["Positively Rated"] = np.where(df["Rating"] > 3,1,0)
df.head()

,Product Name,Brand Name,Price,Rating,Reviews,Review Votes,Positively Rated
0,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,I feel so LUCKY to have found this used (phone...,1.0,1
1,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,"nice phone, nice up grade from my pantach revu...",0.0,1
2,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,5,Very pleased,0.0,1
3,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,It works good but it goes slow sometimes but i...,0.0,1
4,"""CLEAR CLEAN ESN"" Sprint EPIC 4G Galaxy SPH-D7...",Samsung,199.99,4,Great phone to replace my lost phone. The only...,0.0,1


Note : We will consider only "Reviews" and "Positively Rated" columns for our analysis, where "Reviews" is X variable and "Positively Rated" is y

In [16]:
df["Positively Rated"].mean()

# our data is skewed to Positive Class

0.7482686025879323

In [19]:
from sklearn.cross_validation import train_test_split

X_train,X_test,y_train,y_test = train_test_split(df["Reviews"],df["Positively Rated"],random_state=0)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(231207,)
(231207,)
(77070,)
(77070,)


In [21]:
print("X train first entry \n {}".format(X_train[0]))

X train first entry 
 I feel so LUCKY to have found this used (phone to us & not used hard at all), phone on line from someone who upgraded and sold this one. My Son liked his old one that finally fell apart after 2.5+ years and didn't want an upgrade!! Thank you Seller, we really appreciate it & your honesty re: said used phone.I recommend this seller very highly & would but from them again!!


In [101]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline

In [36]:
# use CountVectorizer to fit and then transform the text into Bag of words. Note, we can directly use fit_transform method or we
# can first fit and then transform

vect = CountVectorizer()
vect = vect.fit(X_train)

print(vect.get_feature_names()[:100]) #get_feature_names is a method of seeing the words which have been created from training_set

X_train_vect = vect.transform(X_train)  # after fitting, we transform the X_train to CountVectorized object having vectors

['00', '000', '0000', '00000', '000000', '0000000', '0000from', '0001', '0004', '000ma', '000mah', '000mh', '000restricted', '001', '002', '0051', '006', '007', '00am', '00bucks', '00emotional', '00for', '00it', '00k', '00now', '00pm', '00so', '00time', '00us', '01', '01008', '011', '012', '013287002557427', '013435003182980', '014', '0155379', '016', '016g', '016s', '018633051660f', '019s', '02', '02may13', '02mbps', '03', '032g', '032port', '0330', '03pm', '04', '0400', '0412', '04pm', '04th', '05', '050', '0500tkx', '050mms', '050prot', '051', '056', '0572013', '05th', '05the', '05using', '06', '061', '062', '0630', '066', '06pm', '07', '0700', '07am', '07nov2015', '08', '0804245', '0808', '0825', '0829', '087', '087581287', '08in', '08mms', '08this', '09', '0909853', '09on', '0_1439_7', '0_1507_7', '0_print_120716', '0_user_manual', '0a', '0also', '0an', '0b3tbzlidhq7dce1bv05qdefaota', '0bj7255rf1f1a1118w65', '0c', '0cant']


We can see that the vocabulary which we have generated from CountVectorizer is quite messy. It has lots of misspelled words and words having numbers.

In [37]:
print(X_train_vect.shape)

# so for each 231207 vectors, we have a vector of 53216 length having count of words. NOte : any words that didnt appear in
# X_train, but appeared in X_test will be ignored.

(231207, 53216)


In [38]:
# fit a Logistic Regression the model and check its performance

from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression().fit(X_train_vect,y_train)
X_test_vect = vect.transform(X_test)
predictions = log_reg.predict(X_test_vect)

from sklearn.metrics import f1_score, roc_auc_score
score_f1 = f1_score(y_test,predictions,average="binary")
score_roc_auc = roc_auc_score(y_test,predictions)

print("Score from f1 score {}".format(score_f1))
print("Score from roc_auc {}".format(score_roc_auc))

Score from f1 score 0.9675558387435637
Score from roc_auc 0.9268686014350423


In [53]:
### Note : there are 2 things one is tfidfTransformer : which will transform the Countvectorizer object
# 2nd is TfidfVectorizer which takes X_train directly and then automatically Vectorizers and changes it to tfidf 

In [60]:
# Improving the model with tfigfTransformer

vect = CountVectorizer()
X_train_vect = vect.fit_transform(X_train)

tfidf = TfidfTransformer()
X_train_tfidf = tfidf.fit_transform(X_train_vect)   # note : this takes countVectorizer object
print("Shape of X_train after transformation is {}".format(X_train_tfidf.shape))


log_reg = LogisticRegression().fit(X_train_tfidf,y_train)
X_test_vect = vect.transform(X_test)
X_test_tfidf = tfidf.transform(X_test_vect)
predictions = log_reg.predict(X_test_tfidf)

from sklearn.metrics import f1_score, roc_auc_score
score_f1 = f1_score(y_test,predictions,average="binary")
score_roc_auc = roc_auc_score(y_test,predictions)

print("Score from f1 score {}".format(score_f1))
print("Score from roc_auc {}".format(score_roc_auc))

# so we can see after using tfidf, our model's performance has improved

Shape of X_train after transformation is (231207, 53216)
Score from f1 score 0.9663790795484968
Score from roc_auc 0.9276650766291203


In [55]:
# Improving the model with tfidfVectorizer

tfidf = TfidfVectorizer()
X_train_tfidf = tfidf.fit_transform(X_train) 
print("Shape of X_train after transformation is {}".format(X_train_tfidf.shape))


log_reg = LogisticRegression().fit(X_train_tfidf,y_train)
X_test_tfidf = vect.transform(X_test)
predictions = log_reg.predict(X_test_tfidf)

from sklearn.metrics import f1_score, roc_auc_score
score_f1 = f1_score(y_test,predictions,average="binary")
score_roc_auc = roc_auc_score(y_test,predictions)

print("Score from f1 score {}".format(score_f1))
print("Score from roc_auc {}".format(score_roc_auc))

# so we can see after using tfidf, our model's performance has improved

Shape of X_train after transformation is (231207, 53216)
Score from f1 score 0.9551256095628752
Score from roc_auc 0.9255079702302214


In [61]:
# We can specify the features which we want to choose having some minimum documents. This can be done both in the CountVectorizer
# and tfidf.Suppose we want to choose only those words, that have occured at least 5 times in each sentence etc.

# also, we can tell it to remove stopwords

tfidf = TfidfVectorizer(min_df=5,stop_words="english")
X_train_tfidf = tfidf.fit_transform(X_train)   # note : this takes countVectorizer object
print("Shape of X_train after transformation is {}".format(X_train_tfidf.shape))


log_reg = LogisticRegression().fit(X_train_tfidf,y_train)
X_test_tfidf = tfidf.transform(X_test)
predictions = log_reg.predict(X_test_tfidf)

from sklearn.metrics import f1_score, roc_auc_score
score_f1 = f1_score(y_test,predictions,average="binary")
score_roc_auc = roc_auc_score(y_test,predictions)

print("Score from f1 score {}".format(score_f1))
print("Score from roc_auc {}".format(score_roc_auc))

# so we can see after using tfidf, our model's performance has improved

Shape of X_train after transformation is (231207, 17951)
Score from f1 score 0.9658710145428168
Score from roc_auc 0.9266100666746837


We can see that our features have decreaed from 53216 to 17951 by using min_df=5

In [ ]:
# If you want to see the feature_names, then fit the X_train first and the created object will have feature names. If you are 
# using fit_transform directly, then it is not possible.

In [78]:
tfidf = TfidfVectorizer(min_df=5,stop_words="english")
vect = tfidf.fit(X_train)
X_train_tfidf = vect.transform(X_train)   
print("Shape of X_train after transformation is {}".format(X_train_tfidf.shape))


log_reg = LogisticRegression().fit(X_train_tfidf,y_train)
X_test_tfidf = tfidf.transform(X_test)
predictions = log_reg.predict(X_test_tfidf)

from sklearn.metrics import f1_score, roc_auc_score
score_f1 = f1_score(y_test,predictions,average="binary")
score_roc_auc = roc_auc_score(y_test,predictions)

print("Score from f1 score {}".format(score_f1))
print("Score from roc_auc {}".format(score_roc_auc))

Shape of X_train after transformation is (231207, 17649)
Score from f1 score 0.9611076647366971
Score from roc_auc 0.9146923344156279


In [81]:
feature_names = np.array(vect.get_feature_names())

sorted_coef_index = log_reg.coef_[0].argsort()    # it is sorting as per arguments, so 0 i.e. negative reviews will come first n
                                                   # then positive review words

print("Small coef words or Negative review words \n {}".format(feature_names[sorted_coef_index[:10]]))
print("Large coef words or Positive review words \n {}".format(feature_names[sorted_coef_index[-11:-1]]))

Small coef words 
 ['worst' 'waste' 'disappointed' 'useless' 'return' 'terrible' 'poor'
 'horrible' 'worthless' 'returning']
Large coef words 
 ['best' 'easy' 'far' 'perfectly' 'awesome' 'loves' 'amazing' 'perfect'
 'excellent' 'great']


In [76]:
# an issue with the count_vectorizer and tfidf_vectorizer is that it doesnt maintain the order. This means that any word
# appearing in the beginning or end will be considered as same for eg:

print(log_reg.predict(tfidf.transform(["not an issue,phone is working", "an issue, phone is not working fine "])))

[0 0]


So we can see that in first sentence, it was a positive word, but due to non-maintainence of order, the model considered it as negative

Solution : We can use n_grams model for this. Right now, we are using bag of words model, which is n_gram model with n=1. If we use n=2, the model will not choose single words, but will use 2 words together

In [95]:
tfidf = TfidfVectorizer(ngram_range=(1,3),stop_words="english",min_df=5)
vect = tfidf.fit(X_train)
X_train_tfidf = vect.transform(X_train)
print("Shape of X_train after transformation is {}".format(X_train_tfidf.shape))


log_reg = LogisticRegression().fit(X_train_tfidf,y_train)
X_test_tfidf = tfidf.transform(X_test)
predictions = log_reg.predict(X_test_tfidf)

from sklearn.metrics import f1_score, roc_auc_score
score_f1 = f1_score(y_test,predictions,average="binary")
score_roc_auc = roc_auc_score(y_test,predictions)

print("Score from f1 score {}".format(score_f1))
print("Score from roc_auc {}".format(score_roc_auc))

# we can see that now our vector size has increased to 284382

# hr@tidyquant.com

Shape of X_train after transformation is (231207, 284382)
Score from f1 score 0.9711319583362076
Score from roc_auc 0.9354505150919952


In [97]:
feature_names = np.array(vect.get_feature_names())

sorted_coef_index = log_reg.coef_[0].argsort()


print("Small coef words or Negative review words \n {}".format(feature_names[sorted_coef_index[:10]]))
print("Large coef words or Positive review words \n {}".format(feature_names[sorted_coef_index[-11:-1]]))

Small coef words or Negative review words 
 ['awful phone phone' 'able activated recommend' 'black screen screen'
 'bean easily updated' 'ago sent' 'assignment mismatch location'
 'batter power waiting' 'better issues phone' 'awhile make leap'
 'awkward calling contact']
Large coef words or Positive review words 
 ['2016 lines' 'apps gmail instagram' 'accents aluminum' '20 35 mins'
 '13 16 wow' 'acquire warranty situation' 'android phones wanted'
 'apps easily phone' 'accidentally dropping pantechs' 'android market pros']


In [100]:
print(log_reg.predict(tfidf.transform(["not an issue,phone is working", "an issue, phone is not working fine "])))

# now, we can see that it has correctly detected

[1 1]


In [54]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.pipeline import Pipeline

# clf = Pipeline([("vect",CountVectorizer()),("tfidf" , TfidfTransformer()),("")])